Experimentation with spark while watching Apache Spark Essential Training on lynda.com. Using the NYC taxi data.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/dev/shm/test.csv', index_col=False)

In [3]:
df.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [4]:
df = spark.read.csv('/dev/shm/test.csv', header=True, inferSchema=True).repartition(6).cache()
df.limit(10).show()

+---------+---------+-------------------+---------------+------------------+------------------+------------------+-----------------+------------------+
|       id|vendor_id|    pickup_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude| dropoff_latitude|store_and_fwd_flag|
+---------+---------+-------------------+---------------+------------------+------------------+------------------+-----------------+------------------+
|id1952549|        1|2016-06-18 12:01:37|              1|-74.00577545166016|40.745174407958984|-73.99500274658203|40.75532531738281|                 N|
|id2994195|        1|2016-06-16 12:34:41|              1|-73.98572540283203| 40.76766586303711|-73.97952270507812| 40.7531623840332|                 N|
|id1460514|        2|2016-06-14 22:23:13|              1|-73.95568084716797| 40.76395797729492|-73.97579193115234|40.76530456542969|                 N|
|id0998989|        1|2016-06-28 13:27:55|              4|-74.00822448730469| 40.72438430

In [5]:
df.count()

625134

In [6]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)



In [7]:
df.first()

Row(id='id1952549', vendor_id=1, pickup_datetime=datetime.datetime(2016, 6, 18, 12, 1, 37), passenger_count=1, pickup_longitude=-74.00577545166016, pickup_latitude=40.745174407958984, dropoff_longitude=-73.99500274658203, dropoff_latitude=40.75532531738281, store_and_fwd_flag='N')

In [8]:
df.filter(df['id'] == 'id3220489').show()

+---------+---------+-------------------+---------------+------------------+-----------------+------------------+------------------+------------------+
|       id|vendor_id|    pickup_datetime|passenger_count|  pickup_longitude|  pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+------------------+------------------+
|id3220489|        2|2016-06-21 17:55:29|              1|-73.97895050048828|40.74456024169922|-73.94608306884766|40.775489807128906|                 N|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+------------------+------------------+



In [9]:
df.groupBy('passenger_count').count().orderBy('passenger_count').show()

+---------------+------+
|passenger_count| count|
+---------------+------+
|              0|    23|
|              1|443447|
|              2| 90027|
|              3| 25686|
|              4| 12017|
|              5| 33411|
|              6| 20521|
|              9|     2|
+---------------+------+



In [10]:
df2 = df.select(df['id'], (df['passenger_count']*df['passenger_count']).alias('foo')).groupBy('id').sum()
df2.limit(10).show()

+---------+--------+
|       id|sum(foo)|
+---------+--------+
|id2271966|       1|
|id2024223|       1|
|id1265117|       4|
|id0441361|       1|
|id3220489|       1|
|id3294219|       1|
|id1849118|       1|
|id2826809|       4|
|id3639984|       1|
|id3479561|       1|
+---------+--------+



In [11]:
df.write.saveAsTable('test_sparktable')